In [3]:
from databricks.connect import DatabricksSession
from databricks.sdk.core import Config

runtime_131 = '0601-182128-dcbte59m'
runtime_132 = '0630-162806-47pefcs5'

# Use DEFAULT configuration and pass an existing cluster_id
# Cluster Runtime > 13.2 
config = Config(
    profile = 'DEFAULT',
  cluster_id = runtime_132
)

spark = DatabricksSession.builder.sdkConfig(config).getOrCreate()

# Create a new Delta Table with Universal (Iceberg support) 
[Official Documentation](https://docs.delta.io/3.0.0rc1/delta-uniform.html)

In [4]:
catalog =  'rohitb_play_area'
schema = 'sandbox'

spark.sql(f"use catalog {catalog}")
spark.sql(f"use schema {schema}")

DataFrame[]

In [14]:
# Universal Format is only supported on Unity Catalog tables.
sample_table_name = 'iceberg_test'
spark.sql(f"drop table if exists {sample_table_name}")

spark.sql(f"""
    CREATE OR REPLACE TABLE {sample_table_name}(name STRING, age INT) 
    USING DELTA 
    TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg')
""")
spark.sql(f"insert into {sample_table_name} values ('Amitabh', 73)")
spark.sql(f"insert into {sample_table_name} values ('Dilip', 85)")
spark.sql(f"insert into {sample_table_name} values ('Dev', 53)")

spark.sql(f"select * from {sample_table_name}").show()

+-------+---+
|   name|age|
+-------+---+
|  Dilip| 85|
|Amitabh| 73|
|    Dev| 53|
+-------+---+



In [17]:
spark.sql(f"describe extended {sample_table_name}").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                name|              string|   null|
|                 age|                 int|   null|
|                    |                    |       |
|# Delta Statistic...|                    |       |
|        Column Names|col-2f585ded-2426...|       |
|Column Selection ...|            first-32|       |
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|    rohitb_play_area|       |
|            Database|             sandbox|       |
|               Table|        iceberg_test|       |
|        Created Time|Fri Jun 30 19:33:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|              Spark |       |
|                Type|             MANAGED|       |
|            Location|s3://databricks-e...|       |
|           

# Convert an existing Delta Table to support Uniform

## Download sample data from Kaggle

In [ ]:
import os

# os.environ['KAGGLE_USERNAME'] = "..."
# os.environ['KAGGLE_KEY'] = "..."

import kaggle

kaggle.api.dataset_download_files('stefanoleone992/fifa-20-complete-player-dataset', path='./data', unzip=True)

In [ ]:
spark.sql("""
ALTER TABLE rohitb_play_area.sandbox.transactions SET TBLPROPERTIES(
  'delta.minReaderVersion' = '2',
  'delta.minWriterVersion' = '7')
  """)

In [ ]:
spark.sql("""
ALTER TABLE rohitb_play_area.sandbox.transactions SET TBLPROPERTIES(
  'delta.columnMapping.mode' = 'name',
  'delta.universalFormat.enabledFormats' = 'iceberg')
  """)

In [ ]:
with open("/Users/rohit.bhagwat/.kaggle/kaggle.json") as file:
    data = file.read()

print(data)